In [1]:
from transformers import ViTImageProcessor, BertTokenizer, VisionEncoderDecoderModel
from datasets import load_dataset

2024-10-04 13:25:03.537807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 13:25:03.537939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 13:25:03.671871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from transformers.models.vit import ViTImageProcessor


In [3]:
from transformers import BertTokenizer


In [4]:
from transformers import (
    AutoFeatureExtractor, 
    AutoTokenizer, 
    VisionEncoderDecoderModel,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer, 
    default_data_collator,
)


In [5]:
# from transformers import BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel

# image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# # ViT and BERT configuration
# config_encoder = ViTConfig()
# config_decoder = BertConfig()

# # Set the decoder_start_token_id attribute
# config_decoder.decoder_start_token_id = tokenizer.bos_token_id

# # Vision Encoder-Decoder configuration
# config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# # Set padding-related configurations
# config.pad_token_id = tokenizer.pad_token_id  # pad token ID
# config.pad_token = tokenizer.pad_token  # pad token
# # config.decoder_start_token_id = tokenizer.decoder_start_token_id
# # Instantiate the Vision Encoder-Decoder model with the configured settings
# model = VisionEncoderDecoderModel(config=config)
# model.config.decoder_start_token_id = tokenizer.pad_token_id


In [6]:
from transformers import BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel

image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# ViT and BERT configuration
config_encoder = ViTConfig()
config_decoder = BertConfig()

# Set the decoder_start_token_id attribute
# config_decoder.decoder_start_token_id = tokenizer.bos_token_id

# Vision Encoder-Decoder configuration
config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Set padding-related configurations
config.pad_token_id = tokenizer.pad_token_id  # pad token ID
config.pad_token = tokenizer.pad_token  # pad token
# config.decoder_start_token_id = tokenizer.decoder_start_token_id
# Instantiate the Vision Encoder-Decoder model with the configured settings
model = VisionEncoderDecoderModel(config=config)



preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

In [8]:
import pandas as pd 
df2 = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv')
df1 = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv')

In [9]:
images_captions_df = pd.DataFrame({'imgs': [],
                                    'captions': []})
for i in range(len(df2)):
    uid = df2.iloc[i]['uid']
    image = df2.iloc[i]['filename']
    index = df1.loc[df1['uid'] ==uid]
    
    if not index.empty:    
        index = index.index[0]
        caption = df1.iloc[index]['findings']
        if type(caption) == float:
         
            continue 
        images_captions_df = pd.concat([images_captions_df, pd.DataFrame([{'imgs': image, 'captions': caption}])], ignore_index=True)
images_captions_df.head()

,imgs,captions
0,1_IM-0001-4001.dcm.png,The cardiac silhouette and mediastinum size ar...
1,1_IM-0001-3001.dcm.png,The cardiac silhouette and mediastinum size ar...
2,2_IM-0652-1001.dcm.png,Borderline cardiomegaly. Midline sternotomy XX...
3,2_IM-0652-2001.dcm.png,Borderline cardiomegaly. Midline sternotomy XX...
4,4_IM-2050-1001.dcm.png,There are diffuse bilateral interstitial and a...


In [10]:
p = '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/'
images_captions_df['imgs'] = p+ images_captions_df['imgs']
images_captions_df.head()

,imgs,captions
0,/kaggle/input/chest-xrays-indiana-university/i...,The cardiac silhouette and mediastinum size ar...
1,/kaggle/input/chest-xrays-indiana-university/i...,The cardiac silhouette and mediastinum size ar...
2,/kaggle/input/chest-xrays-indiana-university/i...,Borderline cardiomegaly. Midline sternotomy XX...
3,/kaggle/input/chest-xrays-indiana-university/i...,Borderline cardiomegaly. Midline sternotomy XX...
4,/kaggle/input/chest-xrays-indiana-university/i...,There are diffuse bilateral interstitial and a...


In [11]:
from PIL import Image
import requests
import torch
max_length=512
class CaptionDataset(torch.utils.data.Dataset):
    def __init__(self, images_captions_df, image_processor, tokenizer):
        self.images = images_captions_df['imgs'].values
        self.captions = images_captions_df['captions'].values
        self.image_processor = image_processor
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = str(self.images[idx])
        image = Image.open(image_path)
        image = Image.open(image_path).convert("RGB")
        image_features = self.image_processor(image, return_tensors="pt").pixel_values
        labels = self.tokenizer(self.captions[idx], return_tensors="pt",
                                padding='max_length',
                                return_token_type_ids=True,
                                truncation=True,
#                                 labels=self.captions[idx],
#                                 decoder_start_token_id=self.tokenizer.bos_token_id
                               ).input_ids
#         return {'pixel_values': image_features.squeeze(0), 'input_ids': labels['input_ids'].squeeze(0)}
    
        return {'pixel_values': image_features.squeeze(0), 'labels': labels.squeeze(0),'decoder_start_token_id':2}


In [12]:
from sklearn.model_selection import train_test_split
train_,test_df =train_test_split(images_captions_df, test_size=0.10, shuffle=True, random_state=42)

In [13]:
train_df,val_df =train_test_split(train_, test_size=0.10, shuffle=True, random_state=42)

In [14]:
datsets_train = CaptionDataset(train_df,image_processor, tokenizer)
dataset_test = CaptionDataset(test_df,image_processor, tokenizer)
dataset_val = CaptionDataset(val_df,image_processor, tokenizer)

In [15]:
from transformers import Trainer, TrainingArguments
import os
os.environ["WANDB_MODE"] = "disabled"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
per_device_train_batch_size=10
model=model.to(device)
model.train()
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=datsets_train,         # training dataset
    eval_dataset=dataset_val            # evaluation dataset
)


In [16]:
trainer.train()


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss
10,10.472300
20,10.427200
30,10.363700
40,10.301400
50,10.247100
60,10.174400
70,10.084900
80,9.971600
90,9.851900
100,9.703200


Removed shared tensor {'decoder.cls.predictions.decoder.bias', 'decoder.cls.predictions.decoder.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


TrainOutput(global_step=2620, training_loss=1.1077128774337186, metrics={'train_runtime': 6012.6318, 'train_samples_per_second': 4.357, 'train_steps_per_second': 0.436, 'total_flos': 4.741982584050217e+18, 'train_loss': 1.1077128774337186, 'epoch': 5.0})

### Eval

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.21335908770561218, 'eval_runtime': 98.8495, 'eval_samples_per_second': 5.898, 'eval_steps_per_second': 0.597, 'epoch': 1.0}


dataset_test

In [33]:
type(dataset_test)

__main__.CaptionDataset

In [34]:
import tqdm
predicted_captions = []
for i in tqdm.tqdm( images_captions_df['imgs']):

    img =  Image.open(i).convert("RGB")
    caption = tokenizer.decode(model.generate(feature_extractor(img, return_tensors="pt").pixel_values.to("cuda"))[0],skip_special_tokens=True)
    predicted_captions.append(caption)
print(len(predicted_captions))


  0%|          | 0/6469 [00:00<?, ?it/s]


NameError: name 'feature_extractor' is not defined

In [46]:
from transformers import ViTImageProcessor, BertTokenizer, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load the trained model and tokenizer
model = VisionEncoderDecoderModel.from_pretrained('/kaggle/working/results/checkpoint-2500')  # Update the path to your trained model directory
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to generate caption for a single image
def generate_caption(image_path, model, image_processor, tokenizer, max_length=512):
    # Open and process the image
    image = Image.open(image_path).convert("RGB")
    pixel_values = image_processor(image, return_tensors="pt").pixel_values
    
    # Move the pixel values to the same device as the model
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    pixel_values = pixel_values.to(device)
    model = model.to(device)
    
    # Explicitly pass the decoder_start_token_id during generation
    output_ids = model.generate(
        pixel_values, 
        max_length=max_length, 
        num_beams=4, 
        early_stopping=True,
        decoder_start_token_id=model.config.decoder_start_token_id  # Ensure this is passed directly
    )
    
    # Decode the generated captions into text
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return caption

# Test on a single image
test_image_path = '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/1000_IM-0003-1001.dcm.png'  # Update this path to your image
caption = generate_caption(test_image_path, model, image_processor, tokenizer)

print(f"Generated Caption: {caption}")


Generated Caption: the heart is normal in size. the mediastinum is unremarkable. the lungs are clear.
